# CDS services in Jupyter notebooks
Welcome to a Jupyter notebook demonstrating how to access some CDS services with Python. 

##### How it works: 
Jupyter notebooks offer an interactive way to use Python. If you are viewing this notebook with Binder you can execute every cell of code either by pushing the little 'play' <img src="images/jupyter_run-cell.png" alt="the Play button" style="width:70px; display: inline-block;"/>
symbol on the left of the cell (appears when the mouse pointer moves in its vicinity) or by selecting a cell and pressing 'Shift' + 'Enter' at the same time. Note that cells that have not yet been executed have no number assigned to them, later on they are assigned the number of the order in which they were executed. If they are run again, they get a new number. 

##### Goals:
- gather information on a specific object using <ins>Simbad</ins>, 
- visualise it through survey images using the <ins>Aladin Lite</ins> widget, 
- find and download a catalogue from <ins>Vizier</ins>,
- overlay the sources of this catalogue on the sky view in the Aladin Lite widget, 
- cross-match the sources with a large catalogue using <ins>X-match</ins>. 

##### Required packages: 
 - [`ipyaladin`](https://github.com/cds-astro/ipyaladin), allows to load Aladin Lite in a Jupyter notebook.
 - The `Simbad`, `Vizier` and `XMatch` modules from the [`astroquery`](https://astroquery.readthedocs.io/en/latest/index.html#introduction) package. 
 - [`astropy`](https://docs.astropy.org/en/stable/index.html) is the astronomy Python package.

These packages are ready to use on Binder, while if you run the notebook on your machine, you can install them using `pip` with the command `pip install [name of the package]`.

Let's import the packages by running the following cell:

In [ ]:
import ipyaladin.aladin_widget as ipyal
from astroquery.simbad import Simbad
from astroquery.vizier import Vizier
from astroquery.xmatch import XMatch
import astropy.units as u

### Task #1: Gather information on NGC4038. 
As a first step, we check what information is directly available from the <ins>Simbad</ins> python module:

In [ ]:
Simbad.list_votable_fields()

We get a list of the available fields that we can query and obtain in a table. If we are uncertain about the meaning of a field, we can get a description with `Simbad.get_field_description('field_name')`.
Scrolling the output of the cell above we also learn that the currently active fields are the main_id and the coordinates. If we want to get more information, we need to request it specifically. We can do so by running `Simbad.add_votable_fields('field_name')` before querying the object:

In [ ]:
Simbad.get_field_description('biblio')

In [ ]:
Simbad.reset_votable_fields()
Simbad.add_votable_fields('otype', 'dim', 'dim_bibcode', 'biblio')
info_simbad = Simbad.query_object("NGC4038")
info_simbad 

We now know the object type "galaxy in pair", the object dimensions (major axis, minor axis, inclination angle) with the related bibliographic reference.

Let's search for linked objects: we query sources in a circle of 2 arcmin around NGC4038 of the same object type to find the other galaxy of the pair; then we query sources in the same area with 100% membership probability. 

In [ ]:
siblings = Simbad.query_criteria('region(circle, NGC4038, 2m, 180)', otype='GinPair')
siblings

In [ ]:
members = Simbad.query_criteria('region(circle, NGC4038, 2m, 180)', membership=100)
members

You can learn more on the features of the Simbad python module here: https://astroquery.readthedocs.io/en/latest/simbad/simbad.html

and on the Simbad query criteria: http://simbad.cds.unistra.fr/simbad/sim-fsam

### Task #2: Visualise the source on a sky map
We now open the <ins>Aladin Lite</ins> widget. This widget is supposed to **center** on the Antennae galaxies, have a **field of view** of 0.5deg and show colour images from the DSS **survey**. Typing `aladin` in the second line of code, tells the notebook to display the widget. 

In [ ]:
aladin = ipyal.Aladin(target='Antennae', fov=0.5, survey='P/DSS2/color')
aladin

As with any Aladin Lite implementation, you can interact with this widget. Try to:
 - zoom in and out placing you mouse pointer on top of the image and scrolling. 
 - select other image surveys and manage the current view with <img src="images/ipyaladin_layer.png" alt="the Layer Button" style="width:30px; display: inline-block;"/>.
 - look at another target using the search field <img src="images/ipyaladin_search.png" alt="the Search Button" style="width:30px; display: inline-block;"/>. 

These interactions can also be steered by changing properties of the variable `aladin`. For example, to focus on M101 instead of the Antennae galaxies:

`aladin.target = 'M101'`

### Task #3: Find  additional data
We want to look for available data related to HII regions in these interacting galaxies. We make a query in <ins>Simbad</ins> for HII regions within 14 arcmin around the Antennae galaxies. 

In [ ]:
table_simbad = Simbad.query_criteria('region(circle, Antennae, 14m)', maintype='HII')
table_simbad.show_in_notebook()

Once we have identified a bibliographic reference of interest, we can search the corresponding catalogs on <ins>VizieR</ins>.

Remember that by default VizieR will only return 50 entries (as in the web interface). To get all sources, set `ROW_LIMIT=-1`.

A good practice is to check first what data are available by querying only a small subsample, then customize your Vizier query with the useful rows and columns before downloading the entire catalog.

In [ ]:
cat_viz = Vizier.get_catalogs('2015MNRAS.451.1307Z') 
cat_viz

In the example above, we see that 2 tables are associated with the reference 2015MNRAS.451.1307Z: a table called 'HIIreg' accessible by the index [0] and a table called 'list'accessible by the index [1]. 

Let's check the 'HIIreg' table:

In [ ]:
cat_viz[0].show_in_notebook()

Now we can customize our VizieR query by selecting columns, filtering over the interacting galaxies (n_Name='int') and keeping only the first region for each galaxy (Seq=1, for visualisation purposes).

In [ ]:
myViz = Vizier(columns=['Name', 'RAJ2000', 'DEJ2000', 'logLHa', 'R', 'sigma'],
            column_filters={'Seq': '==1', 'n_Name': 'int'})

myViz.ROW_LIMIT = -1

cat_hii = myViz.get_catalogs('2015MNRAS.451.1307Z')[0]
cat_hii.show_in_notebook()

We can now display the table on <ins>Aladin</ins>. 
This should be obtained by simply typing `aladin.add_table(cat_hii)`
but if you get errors on the format of some columns, 
you can just send name and coordinates to Aladin:

In [ ]:
aladin.add_table(cat_hii['Name','RAJ2000','DEJ2000'])

Go up to the <ins>Aladin Lite</ins> widget, zoom out to see all the sources, then rotate and zoom in to see the interacting galaxies. You can click on one source to display the information in the table. 

Change the view to the AllWISE Infrared survey, either in the widget:

`Base image layer -> The Wide-field Infrared Survey Explorer` 

or in the command line below.

You can find the full list of available sky maps at https://aladin.u-strasbg.fr/hips/list

In [ ]:
aladin.survey='P/allWISE/color'

If you want to overlay the survey instead of replacing it:

`aladin.overlay_survey= 'P/allWISE/color'`

`aladin.overlay_survey_opacity = 0.5`

#### Exercise: 
Retrive the second table from Zaragoza-Cardiel+, 2015 (2015MNRAS.451.1307Z) containing data related to the image fits files and try to answer the questions:
- What are the typical sizes of the fits images?
- What is the approximate resolution of the images? [arcsec/pix]

In [ ]:
#write your code here


### Task #5: Cross-match tables

We have HII data for a sample of interacting galaxies. Since we are also interested in the infrared fluxes we search for the AllWISE data release. 

We want to match the sources in J/MNRAS/451/1307/HIIreg with the AllWISE source catalogue which contains 747 million objects.

We use the <ins>X_Match</ins> python module for cross-identification of objects which is particularly efficient and fast with very large catalogues, like AllWISE. All we need are the names of the catalogues, the names of the columns containing the coordinates of the sources, and the desired maximum distance for the match (optionally the area as well, otherwise all-sky is the default).

In [ ]:
#What is the AllWISE catalogue reference in VizieR?
cat_allwise = Vizier.get_catalogs('AllWISE')
cat_allwise

In [ ]:
xmatch_zbf_wise = XMatch.query(cat1='vizier:J/MNRAS/451/1307/HIIreg', 
                               cat2='vizier:II/328/allwise',
                               max_distance=10 * u.arcsec, 
                               colRA1='RAJ2000', colDec1='DEJ2000',
                               colRA2='RAJ2000', colDec2='DEJ2000')
xmatch_zbf_wise.show_in_notebook()

<h4 align="center">This is the end of the tutorial.</h4> 

<h4 align="center">Try to apply what you have learned here to your data sets and science cases! </h4>